### Dataset

In [46]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop, SGD
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from keras.losses import categorical_crossentropy
from sklearn.metrics import log_loss
from keras.utils import to_categorical

In [47]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [48]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

### Armado y entrenamiento de un clasificador a partir de los datos originales

#### Modelo LSTM

In [49]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
classifier_dataset['Open_time'] = pd.to_datetime(classifier_dataset['Open_time'])
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-5:]
classifier_dataset = classifier_dataset[:-5]

In [50]:
display(classifier_dataset.tail())

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
898,2024-03-13,10.74,11.46,10.64,11.37,276468.00,73072.41,52659.71,2501197.00,4004.79,482305.78,1536498.00,630.50,2526002.56,1265237.00,9.45,9.70,11.82,9.45,7.07,73.43,64.00,81.00,92576.00,275,205,1.00,73.00,578.00,553.00,164000.00,Alcista
899,2024-03-14,11.37,11.89,10.68,11.56,536988.00,71388.94,71757.63,2994869.00,3881.70,648237.52,1919963.00,603.20,2119540.30,1038297.00,9.65,9.88,12.03,9.65,7.27,74.51,102.00,133.00,145727.00,211,181,4.00,29.00,754.00,677.00,327000.00,Alcista
900,2024-03-15,11.56,11.71,9.97,10.81,557152.00,69499.85,103334.04,3904445.00,3742.19,947537.41,2487337.00,632.70,3066312.79,1365283.00,9.80,9.97,12.07,9.80,7.52,63.27,88.00,83.00,147460.00,238,106,0.00,25.00,493.00,430.00,360000.00,Bajista
901,2024-03-16,10.81,10.90,9.50,9.68,330505.00,65300.63,55926.95,2729019.00,3523.09,548288.16,1798939.00,576.40,1811838.04,1025452.00,9.89,9.94,11.99,9.89,7.78,50.82,30.00,49.00,88095.00,670,471,0.00,20.00,513.00,403.00,209000.00,Bajista
902,2024-03-17,9.68,10.25,9.19,10.08,229683.00,68393.48,49742.22,2449156.00,3644.71,517790.99,1721355.00,571.70,1712920.34,802297.00,9.98,9.95,11.93,9.98,8.04,54.27,36.00,48.00,71390.00,693,413,0.00,21.00,295.00,277.00,150000.00,Alcista


In [51]:
classifier_dataset.shape

(903, 32)

In [52]:
X = classifier_dataset.drop(columns=["Tendencia", "Open_time"])
y = classifier_dataset["Tendencia"]

# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(classifier_dataset["Tendencia"])

y = y.to_numpy().reshape(-1, 1)
onehot_encoder = OneHotEncoder(sparse=False)
y_one_hot = onehot_encoder.fit_transform(y)


In [53]:
display(y_one_hot)

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [54]:
import warnings
warnings.filterwarnings('ignore')

def create_model(activation, units, dropout, learning_rate, l2_penalty, depth, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=int(units/2), activation=activation, input_shape=(len(X.columns), 1), return_sequences=True, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    for _ in range(depth - 1):
        model.add(LSTM(units=units, activation=activation, return_sequences=True, kernel_regularizer=l2(l2_penalty)),)
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    
    model.add(LSTM(units=int(units*2), activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(units=3, activation='softmax'))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
regressor = KerasRegressor(build_fn=create_model, verbose=0, activation='relu', units=50, dropout=0.2, learning_rate=0.1, l2_penalty=0.001, depth=2, optimizer='adam')

cv = TimeSeriesSplit(n_splits=10).split(X)
param_space = {
    'depth': [2, 3, 4, 5],
    'activation': ['relu', 'tanh', 'swish', 'selu'],
    'units': [64, 128, 256, 512],
    'dropout': [0.1, 0.2, 0.3, 0.4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'l2_penalty': [0.001, 0.01, 0.1]
}

def categorical_crossentropy_loss(estimator, X_test, y_test):
    y_pred = estimator.predict(X_test)
    
    if np.isnan(y_pred).any():
        y_pred[np.isnan(y_pred)] = 0
    
    loss = log_loss(y_test, y_pred)
    return loss

bayes_search = BayesSearchCV(regressor, param_space, scoring=categorical_crossentropy_loss, cv=cv, verbose=0)#10)
bayes_result = bayes_search.fit(X, y_one_hot, callbacks=[early_stopping])

In [55]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y_one_hot)

Best score: 8.120184086861974
Best parameters: OrderedDict([('activation', 'selu'), ('batch_size', 128), ('depth', 5), ('dropout', 0.2), ('epochs', 10), ('l2_penalty', 0.01), ('learning_rate', 0.001), ('optimizer', 'adam'), ('units', 512)])


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x7fb76a0a71f0>
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=128
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	activation=selu
	units=512
	dropout=0.2
	learning_rate=0.001
	l2_penalty=0.01
	depth=5
)

In [56]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('lstm_classifier/top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 32), ('depth', 4), ('dropout', 0.1), ('epochs', 100), ('l2_penalty', 0.1), ('learning_rate', 0.01), ('optimizer', 'rmsprop'), ('units', 128)])
Puntaje: 1.1057941069737893
Modelo 2
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 32), ('depth', 5), ('dropout', 0.3), ('epochs', 30), ('l2_penalty', 0.1), ('learning_rate', 0.0001), ('optimizer', 'sgd'), ('units', 64)])
Puntaje: 1.3284329727438409
Modelo 3
Hiperparámetros: OrderedDict([('activation', 'swish'), ('batch_size', 32), ('depth', 4), ('dropout', 0.4), ('epochs', 100), ('l2_penalty', 0.01), ('learning_rate', 0.001), ('optimizer', 'sgd'), ('units', 256)])
Puntaje: 2.057655166026172
Modelo 4
Hiperparámetros: OrderedDict([('activation', 'swish'), ('batch_size', 64), ('depth', 3), ('dropout', 0.3), ('epochs', 10), ('l2_penalty', 0.1), ('learning_rate', 0.001), ('optimizer', 'sgd'), ('units', 128)])
Puntaje: 1.4302739

#### Armado del ensamble

In [57]:
def generate_prime_seeds(n):
    seeds = []
    num = 70001  # Comenzamos desde el primer número primo mayor que 70000
    while len(seeds) < n:
        is_prime = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            seeds.append(num)
        num += 1
    return seeds


In [58]:
## Clase personalizada para hacer el ensamble, dado que sklearn no provee ninguna clase que permita hacer ensmble
## de modelos re regresion multivariados
class MultivariableVotingRegressor:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def predict(self, X):
        # Hacer predicciones con cada modelo
        predictions = np.array([model.predict(X) for model in self.models])
        
        # Calcular la moda de las predicciones
        mode_predictions = np.argmax(np.sum(predictions, axis=0), axis=1)
        
        return mode_predictions


In [59]:

import json

# Leer los hiperparámetros desde el archivo JSON
with open('lstm_classifier/top_5_hyperparameters.json', 'r') as f:
    top_hyperparameters = json.load(f)

prime_seeds = generate_prime_seeds(300)
models = []
best_seeds= {}

# Train models with different seeds for each set of hyperparameters
for mode_number, params in enumerate(top_hyperparameters['best_params']):
    best_validation_errors = {}
    
    for seed_number, seed in enumerate(prime_seeds):
        model = KerasRegressor(build_fn=create_model, random_state=seed, verbose=0, **params)
        model.fit(X, y_one_hot)
        
        # Make predictions with the model
        model_predictions = model.predict(X)
        
        # Calculate error (training error)
        train_error = categorical_crossentropy(y_one_hot, model_predictions)
        
        mean_train_error = np.mean(train_error)

        # Update best validation error for this seed
        best_validation_errors[seed] = mean_train_error
    
    # print("Best validation errors:", best_validation_errors)

    # Find the best seed for this set of hyperparameters
    best_seed_for_params = min(best_validation_errors, key=lambda k: best_validation_errors[k])
    best_seeds[str(params)] = best_seed_for_params
    
    # Create and train the model with the best seed
    model = KerasRegressor(build_fn=create_model, random_state=best_seed_for_params, verbose=0, **params)
    model.fit(X, y_one_hot)
    models.append(model)

# Define and train the ensemble model
ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y_one_hot)

# Save the best seeds to a JSON file
with open('lstm_classifier/best_seeds.json', 'w') as f:
    json.dump(best_seeds, f)


#### Clasificacion con el ensamble sobre las redicciones de los modelos generativos

In [ ]:
with open('lstm_classifier/best_seeds.json', 'r') as f:
    best_seeds = json.load(f)

# 21 Crear y entrenar los modelos con los hiperparámetros y semillas guardados
models = []
for params_str, seed in best_seeds.items():
    params = json.loads(params_str.replace("'", "\""))
    model = KerasRegressor(build_fn=create_model, random_state=seed, **params)
    model.fit(X, y_one_hot)
    models.append(model)

Epoch 1/50
29/29 [==============================] - 2s 9ms/step - loss: 43.2445 - accuracy: 0.4031
Epoch 2/50
29/29 [==============================] - 0s 9ms/step - loss: 38.9355 - accuracy: 0.3976
Epoch 3/50
29/29 [==============================] - 0s 10ms/step - loss: 34.9458 - accuracy: 0.4463
Epoch 4/50
29/29 [==============================] - 0s 10ms/step - loss: 31.4929 - accuracy: 0.4264
Epoch 5/50
29/29 [==============================] - 0s 9ms/step - loss: 28.3612 - accuracy: 0.4419
Epoch 6/50
29/29 [==============================] - 0s 9ms/step - loss: 25.6081 - accuracy: 0.4286
Epoch 7/50
29/29 [==============================] - 0s 9ms/step - loss: 23.1227 - accuracy: 0.4319
Epoch 8/50
29/29 [==============================] - 0s 9ms/step - loss: 20.8551 - accuracy: 0.4330
Epoch 9/50
29/29 [==============================] - 0s 9ms/step - loss: 18.9135 - accuracy: 0.4297
Epoch 10/50
29/29 [==============================] - 0s 10ms/step - loss: 17.0915 - accuracy: 0.4297
Epoch 

In [ ]:
ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y_one_hot)

Epoch 1/50
29/29 [==============================] - 2s 12ms/step - loss: 43.2445 - accuracy: 0.4031
Epoch 2/50
29/29 [==============================] - 0s 10ms/step - loss: 38.9355 - accuracy: 0.3976
Epoch 3/50
29/29 [==============================] - 0s 16ms/step - loss: 34.9458 - accuracy: 0.4463
Epoch 4/50
29/29 [==============================] - 0s 12ms/step - loss: 31.4929 - accuracy: 0.4264
Epoch 5/50
29/29 [==============================] - 0s 12ms/step - loss: 28.3612 - accuracy: 0.4419
Epoch 6/50
29/29 [==============================] - 0s 10ms/step - loss: 25.6081 - accuracy: 0.4286
Epoch 7/50
29/29 [==============================] - 0s 11ms/step - loss: 23.1227 - accuracy: 0.4319
Epoch 8/50
29/29 [==============================] - 0s 10ms/step - loss: 20.8551 - accuracy: 0.4330
Epoch 9/50
29/29 [==============================] - 0s 12ms/step - loss: 18.9135 - accuracy: 0.4297
Epoch 10/50
29/29 [==============================] - 0s 11ms/step - loss: 17.0915 - accuracy: 0.4297

##### Datos originales:

In [ ]:
display(clasifier_validation)

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
903,2024-03-18,10.08,10.46,9.60,9.90,245319.00,67609.99,55691.08,2464515.00,3520.46,570901.29,1906387.00,555.40,2284301.81,994512.00,10.06,9.95,11.86,10.06,8.26,52.48,34.00,43.00,84706.00,696,471,0.00,43.00,343.00,228.00,154000.00,Bajista
904,2024-03-19,9.90,9.99,8.60,8.77,341363.00,61937.40,101005.32,3593832.00,3158.64,1049629.69,2647385.00,507.70,2551361.51,1213572.00,10.08,9.84,11.81,10.08,8.35,42.93,120.00,126.00,135180.00,961,509,1.00,56.00,534.00,433.00,221000.00,Bajista
905,2024-03-20,8.77,9.57,8.49,9.48,267797.00,67840.51,90420.59,3549793.00,3516.53,1207322.82,2987953.00,556.80,1425296.58,809335.00,10.14,9.80,11.68,10.14,8.60,49.21,185.00,117.00,112997.00,866,555,1.00,40.00,473.00,386.00,171000.00,Alcista
906,2024-03-21,9.48,9.58,9.07,9.18,156774.00,65501.27,53357.48,2388390.00,3492.85,602755.21,1791989.00,553.80,953921.37,563996.00,10.17,9.74,11.63,10.17,8.71,46.85,64.00,81.00,66543.00,692,533,0.00,24.00,350.00,290.00,101000.00,Bajista
907,2024-03-22,9.18,9.37,8.69,8.94,147578.00,63796.64,51482.38,2492881.00,3336.35,558848.89,1747756.00,553.80,1181298.51,712381.00,10.14,9.67,11.67,10.14,8.62,45.00,57.00,66.00,68616.00,681,546,0.00,41.00,252.00,206.00,92000.00,Bajista


In [ ]:
n_clases = 3 

validation_predictions = ensemble.predict(clasifier_validation.drop(columns=["Open_time", "Tendencia"]))
predicciones_one_hot = to_categorical(validation_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(validation_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 182ms/step


array([1, 1, 1, 1, 1])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista']

##### Datos generados por auto ml con prophet:

In [ ]:
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
auto_mp_prophet_predictions = ensemble.predict(auto_ml_prophet_df.drop(columns=["Open_time"]))

predicciones_one_hot = to_categorical(auto_mp_prophet_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(auto_mp_prophet_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 38ms/step


array([0, 0, 0, 0, 0])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Alcista', 'Alcista', 'Alcista', 'Alcista', 'Alcista']

##### Datos generados por auto ml con stats:

In [ ]:
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
auto_mp_stats_predictions = ensemble.predict(auto_ml_stats_df.drop(columns=["Open_time"]))

predicciones_one_hot = to_categorical(auto_mp_stats_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(auto_mp_stats_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 38ms/step


array([0, 0, 0, 0, 0])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Alcista', 'Alcista', 'Alcista', 'Alcista', 'Alcista']

##### Datos generados por auto ml con modelos clasicos:

In [ ]:
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
auto_mp_predictions = ensemble.predict(auto_ml_df.drop(columns=["Open_time", "Next_Day_Target"]))

predicciones_one_hot = to_categorical(auto_mp_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(auto_mp_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 39ms/step


array([1, 1, 1, 1, 1])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista']